In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train_data.head()

In [ ]:
print(train_data.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras

x_train = train_data.drop(['label'], axis=1)
y_train = train_data['label']
y_train = keras.utils.to_categorical(y_train,num_classes=10)
x_train = np.array(x_train, dtype=np.float32)/255
x_train = x_train.reshape(-1,28,28,1)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(x_train[2], cmap='gray')
print(y_train[2])

In [ ]:
plt.imshow(x_train[4], cmap='gray')
print(y_train[4])

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()


In [ ]:
x_mnist = np.concatenate((x_train_mnist,x_test_mnist))
y_mnist = np.concatenate((y_train_mnist,y_test_mnist))
x_mnist = x_mnist.reshape(-1,28,28,1)
x_mnist = x_mnist.astype(np.float32) / 255
y_mnist = keras.utils.to_categorical(y_mnist,num_classes=10)

In [ ]:
print('x_train==> ',x_train.shape)
print('y_train==> ',y_train.shape)
print('x_mnist==> ',x_mnist.shape)
print('y_mnist==> ',y_mnist.shape)

In [ ]:
X = np.concatenate((x_train,x_mnist))
y = np.concatenate((y_train,y_mnist))

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
def creat_model():
    input_shape = (28,28,1)
    input_layer = Input(input_shape)
    layer = Conv2D(32,(5,5),activation = tf.nn.relu, padding='same', input_shape = input_shape)(input_layer)
    layer = MaxPool2D((2,2))(layer)
    layer = Conv2D(64,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(64,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = MaxPool2D((2,2))(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = MaxPool2D((2,2))(layer)



    flatten = Flatten()(layer)

    layer = Dense(512,activation = tf.nn.relu)(flatten)
    layer = Dropout(0.25)(layer)
    layer = Dense(512,activation = tf.nn.relu)(layer)
    layer  = Dropout(0.25)(layer)
    layer = Dense(512,activation = tf.nn.relu)(layer)
    layer  = Dropout(0.25)(layer)
    output_layer = Dense(10, activation = tf.nn.softmax)(layer)
    model = Model(input_layer,output_layer)
    
    model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
model = creat_model()
model.summary()

In [ ]:
datagen = ImageDataGenerator(rotation_range=20,
                  width_shift_range=0.20,
                  shear_range=15,
                  zoom_range=0.10,
                  validation_split=0.25,
                  horizontal_flip=False)
datagen.fit(X)
generator_train  = datagen.flow(X,y,batch_size=256,subset='training')
generator_validation  = datagen.flow(X,y,batch_size=64,subset='validation')

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5 ,min_lr=0.000001,verbose=1)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='digit-recognizer-model.hdf5',monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=1)

In [ ]:
epoch = 60
model_hist = model.fit(generator_train,validation_data=generator_validation,epochs=epoch,callbacks=[reduce_lr,checkpoint],verbose=1)

In [ ]:
from tensorflow.keras.models import Model

#load best weights
model = creat_model()
model.load_weights('digit-recognizer-model.hdf5')

In [ ]:
x = np.arange(1, epoch + 1)
train_acc = model_hist.history['accuracy']
train_loss = model_hist.history['loss']
val_acc = model_hist.history['val_accuracy']
val_loss = model_hist.history['val_loss']

In [ ]:
plt.plot(x , train_acc, x, val_acc)
plt.show()
plt.plot(x , train_loss, x, val_loss)
plt.show()

In [ ]:
test_data = np.array(test_data, dtype=np.float32)/255
test_data = test_data.reshape(-1,28,28,1)
prediction = model.predict(test_data)


In [ ]:
predict = np.array(np.round(prediction), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
out = [{'ImageId': i+1, 'Label': predict[i][0]} for i in range(len(predict))]
pd.DataFrame(out).to_csv('submission.csv', index=False)